In [1]:
! pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! mkdir ~/.kaggle


In [3]:
! cp kaggle.json ~/.kaggle/


In [4]:
! chmod 600 ~/.kaggle/kaggle.json


In [5]:
#load modules
import os
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [6]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

 86% 13.0M/15.1M [00:01<00:00, 17.9MB/s]
100% 15.1M/15.1M [00:01<00:00, 14.8MB/s]


In [7]:
! unzip competitive-data-science-predict-future-sales.zip


Archive:  competitive-data-science-predict-future-sales.zip
  inflating: item_categories.csv     
  inflating: items.csv               
  inflating: sales_train.csv         
  inflating: sample_submission.csv   
  inflating: shops.csv               
  inflating: test.csv                


In [8]:
#loading data 
# os.listdir('../input')
item_cat = pd.read_csv('/content/item_categories.csv')
items = pd.read_csv('/content/items.csv')
shops = pd.read_csv('/content/shops.csv')
sample_submission = pd.read_csv('/content/sample_submission.csv')

In [9]:
df_train = pd.read_csv('/content/sales_train.csv')
df_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [11]:
df_train.isnull().sum().sum()

0

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [13]:
df_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [14]:
#convert date to datetime format
df_train['date'] = pd.to_datetime(df_train['date'],format = '%d.%m.%Y')

In [15]:
#create pivot table
dataset = df_train.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')
dataset.reset_index(inplace = True)
dataset

shop_id item_id item_cnt_day                        ...        \
date_block_num                            0   1  2  3  4  5  6  7  ... 24 25   
0                    0      30            0  31  0  0  0  0  0  0  ...  0  0   
1                    0      31            0  11  0  0  0  0  0  0  ...  0  0   
2                    0      32            6  10  0  0  0  0  0  0  ...  0  0   
3                    0      33            3   3  0  0  0  0  0  0  ...  0  0   
4                    0      35            1  14  0  0  0  0  0  0  ...  0  0   
...                ...     ...          ...  .. .. .. .. .. .. ..  ... .. ..   
424119              59   22154            1   0  0  0  0  0  0  0  ...  0  0   
424120              59   22155            0   0  0  0  0  0  1  0  ...  0  0   
424121              59   22162            0   0  0  0  0  0  0  0  ...  0  9   
424122              59   22164            0   0  0  0  0  0  0  0  ...  0  2   
424123              59   22167            0   0  0  0  0  0  0  0  ...  0  0   

                                        
date_block_num 26 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  0  
1               0  0  0  0  0  0  0  0  
2               0  0  0  0  0  0  0  0  
3               0  0  0  0  0  0  0  0  
4               0  0  0  0  0  0  0  0  
...            .. .. .. .. .. .. .. ..  
424119          0  0  0  0  0  0  0  0  
424120          0  0  0  0  0  0  0  0  
424121          4  1  1  0  0  1  0  0  
424122          1  2  0  0  1  0  0  0  
424123          0  0  0  0  0  0  0  0  

[424124 rows x 36 columns]

In [16]:
#Load test set
df_test = pd.read_csv('/content/test.csv')
df_test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [17]:
#merge pivot table with test set
dataset = pd.merge(df_test,dataset,on = ['item_id','shop_id'],how = 'left')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [18]:
#check for any null values
dataset.isnull().sum().sum()

3495064

In [19]:
#fill all NaN values with 0
dataset.fillna(0,inplace = True)
dataset.isnull().sum().sum()

0

In [20]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 37 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID                  214200 non-null  int64  
 1   shop_id             214200 non-null  int64  
 2   item_id             214200 non-null  int64  
 3   (item_cnt_day, 0)   214200 non-null  float64
 4   (item_cnt_day, 1)   214200 non-null  float64
 5   (item_cnt_day, 2)   214200 non-null  float64
 6   (item_cnt_day, 3)   214200 non-null  float64
 7   (item_cnt_day, 4)   214200 non-null  float64
 8   (item_cnt_day, 5)   214200 non-null  float64
 9   (item_cnt_day, 6)   214200 non-null  float64
 10  (item_cnt_day, 7)   214200 non-null  float64
 11  (item_cnt_day, 8)   214200 non-null  float64
 12  (item_cnt_day, 9)   214200 non-null  float64
 13  (item_cnt_day, 10)  214200 non-null  float64
 14  (item_cnt_day, 11)  214200 non-null  float64
 15  (item_cnt_day, 12)  214200 non-nul

In [21]:
#drop shop_id and item_id
dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#split the dataset in two
#keep all columns execpt the last one 
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]
# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)
# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)

(214200, 33, 1) (214200, 1) (214200, 33, 1)


In [23]:
# create sequential model
my_model = Sequential()
my_model.add(LSTM(units = 64,input_shape = (33,1)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))
my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [24]:
#fit the model
my_model.fit(X_train,y_train,batch_size = 4096,epochs = 10)

Epoch 1/10
53/53 [==============================] - 7s 16ms/step - loss: 30.6590 - mean_squared_error: 30.6590
Epoch 2/10
53/53 [==============================] - 1s 15ms/step - loss: 30.2758 - mean_squared_error: 30.2758
Epoch 3/10
53/53 [==============================] - 1s 15ms/step - loss: 30.0886 - mean_squared_error: 30.0886
Epoch 4/10
53/53 [==============================] - 1s 15ms/step - loss: 29.9602 - mean_squared_error: 29.9602
Epoch 5/10
53/53 [==============================] - 1s 15ms/step - loss: 29.8511 - mean_squared_error: 29.8511
Epoch 6/10
53/53 [==============================] - 1s 14ms/step - loss: 29.7825 - mean_squared_error: 29.7825
Epoch 7/10
53/53 [==============================] - 1s 15ms/step - loss: 29.7755 - mean_squared_error: 29.7755
Epoch 8/10
53/53 [==============================] - 1s 14ms/step - loss: 29.7670 - mean_squared_error: 29.7670
Epoch 9/10
53/53 [==============================] - 1s 14ms/step - loss: 29.7278 - mean_squared_error: 29.7278
E

In [25]:
# creating submission file 
submission_file = my_model.predict(X_test)
# we will keep every value between 0 and 20
submission_file = submission_file.clip(0,20)
# creating dataframe with required columns 
submission_trp = pd.DataFrame({'ID':df_test['ID'],'item_cnt_month':submission_file.ravel()})
# creating csv file from dataframe
submission_trp.to_csv('submission.csv',index = False)
submission_trp

6694/6694 [==============================] - 15s 2ms/step


,ID,item_cnt_month
0,0,0.415605
1,1,0.115669
2,2,0.733913
3,3,0.152618
4,4,0.115669
...,...,...
214195,214195,0.339353
214196,214196,0.115669
214197,214197,0.116795
214198,214198,0.115669


In [36]:
import joblib

# save the model to disk
filename = 'lstm_model.pkl'
joblib.dump(my_model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)


In [37]:
from google. colab import files
files. download("lstm_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>